# Урок 14. Transfer learning

Взять данные из https://www.kaggle.com/datasets/mrapplexz/bashim-quotes обучить модель GPT для генерации своих цитат

Взять новостные данные из https://github.com/natasha/corus load_lenta2 нам понадобиться сам текст и заголовок обучить модель T5/ или GPT для генерации заголовков для статей

## Задание 1

In [1]:
# !pip install transformers sentencepiece --quiet

In [2]:
import pandas as pd
import json
import torch
import random
from transformers import AutoTokenizer, AutoModelForCausalLM, TextDataset
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import re
    
pd.set_option('display.max_colwidth', None)

def clear_text(text):
    clr_text = re.sub(r"<.*?>", " ", text).lower()
    clr_text = summary = re.sub(r"\s", " ", clr_text)
    return clr_text

def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts).strip()
        summary = re.sub(r"", "", summary)
        summary = re.sub(r"<[\w+,\!, -]>", "", summary)
        summary = re.sub(r"<\w+>", "", summary)
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator
    
DATASET_PATH = './dataset.jsonl'

with open(DATASET_PATH) as f:
     df = pd.read_json(DATASET_PATH, lines=True).set_index('id')

df.drop(['date', 'rating'], axis=1, inplace=True) # эти колонки не нужны
df["clear_text"] = df["text"].apply(lambda x: clear_text(x)) # удаление тегов и лишних пробелов из текста

df.head(2)

2023-11-21 15:40:51.359144: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-21 15:40:52.264887: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


,text,clear_text
id,,
1,"<Ares> ppdv, все юниксы очень дружелюбны.. они просто очень разборчивы в друзьях ;)","ppdv, все юниксы очень дружелюбны.. они просто очень разборчивы в друзьях ;)"
2,"<томатик_рад> а ты не чувствуешь красоту мира?\n<fox> честно говоря, я сейчас чувствую только отсутствие http.\n<томатик_рад> не туда смотришь, глянь вокруг!\n<fox> как я гляну, если http не работает? :/","а ты не чувствуешь красоту мира? честно говоря, я сейчас чувствую только отсутствие http. не туда смотришь, глянь вокруг! как я гляну, если http не работает? :/"


In [3]:
model_name = 'sberbank-ai/rugpt3small_based_on_gpt2' # Обучать будем небольшую модель, чтобы не ждать долго
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [4]:
sep = '\n***\n'
prefix = sep.join([''] + random.sample(list(df['clear_text']), k=2) + [''])
tokens = tokenizer(prefix, return_tensors='pt')
tokens = {k: v.to(model.device) for k, v in tokens.items()}
end_token_id = tokenizer.encode('***')[0]
print(prefix)


***
  ладно, я в евей. пока. * ritsuko[solaris] прочел ``еврей"" и удивился   как ето -- пока?!(tm)
***
kraidiky: когда наступит апокалипсис я буду в ужасе метаться и искать где-нибудь схему газгольдера, но интернетов уже не будет! :) olegkrasnov: зачем вам схема газгольдера? alcanoid: ну надо же где-то хранить выделяемый в состоянии ужаса газ.
***



In [5]:
size = tokens['input_ids'].shape[1]
output = model.generate(
    **tokens, 
    do_sample=False, 
    max_length=size+128, 
    repetition_penalty=4.2, 
    temperature=0.7,
    num_beams=10,
)
decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(result)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Kraidiky: а что это за "газгольдер" на самом деле? kraidiky: Газгольдер для хранения сжиженного природного газа (СПГ).
***
alcanoid: А почему вы так думаете? Kraidiky: потому что СПГ - это жидкий азот.
***
kraidiky: у вас есть какие-то соображения по этому поводу? kraidiky: мне кажется, что с точки зрения здравого смысла эта схема должна выглядеть примерно так: http://www.gazeta.ru/politics/news/


In [6]:
train, test = train_test_split(df.loc[:10000, 'clear_text'],  # обрезали до 10000
                               test_size=0.15)

build_text_files(train,'./train_dataset.txt')
build_text_files(test,'./test_dataset.txt')

print("Train dataset length: "+ str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 1666
Test dataset length: 294


In [7]:
train_path = './train_dataset.txt'
test_path = './test_dataset.txt'

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [8]:
training_args = TrainingArguments(
    output_dir="./gpt2-train", 
    overwrite_output_dir=True, 
    num_train_epochs=3, 
    per_device_train_batch_size=4, 
    per_device_eval_batch_size=4,  
    eval_steps = 400, 
    save_steps=800, 
    warmup_steps=500,
    )

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [10]:
trainer.train() # начинаем дообучать модель

Step,Training Loss


TrainOutput(global_step=480, training_loss=4.334056091308594, metrics={'train_runtime': 49.1347, 'train_samples_per_second': 39.015, 'train_steps_per_second': 9.769, 'total_flos': 125224206336000.0, 'train_loss': 4.334056091308594, 'epoch': 3.0})

In [11]:
# Сохраняем обученную модель
trainer.save_model()
tokenizer.save_pretrained('./gpt2_tokenizer_pretrained')
model.save_pretrained('./gpt2_model_pretrained')

# Загружаем обученную модель
tokenizer = AutoTokenizer.from_pretrained("./gpt2_tokenizer_pretrained")
model_new = AutoModelForCausalLM.from_pretrained("./gpt2_model_pretrained")

In [12]:
# Проверяем генерацию текста
size = tokens['input_ids'].shape[1]
output = model_new.generate(
    **tokens, 
    do_sample=False, 
    max_length=size+100, 
    repetition_penalty=5., 
    temperature=0.5,
    num_beams=10,
)
decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(result)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ritsuko[solaris] задумался о том, что может означать слово "апокалипсис". kraidiky: у меня есть подозрение, что это сокращение от слова "апокалипсис", которое можно перевести как "погибший мир" (в переводе на русский язык оно звучит так - "смерть всему живому"), а также выражение "спасение утопающих - дело рук самих утопающих" (в переводе на русский язык оно звучит так


## Задание 2

In [18]:
# !pip install corus --quiet
# !pip install datasets --quiet
# !wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.1/lenta-ru-news.csv.bz2

SyntaxError: invalid syntax (1798471167.py, line 2)

In [30]:
from corus import load_lenta2
from datasets import Dataset
from transformers import T5ForConditionalGeneration
from datasets import load_from_disk


path = 'lenta-ru-news.csv.bz2'
records = load_lenta2(path)
# next(records)
data = [(record.title, record.text) for record in records]
df_news = pd.DataFrame({'title': [record[0] for record in data], 'text': [record[1] for record in data]})
df_news.head()

,title,text
0,1914. Русские войска вступили в пределы Венгрии,"Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии. «Русский инвалид», 16 сентября 1914 года."
1,1914. Празднование столетия М.Ю. Лермонтова отложено,"Министерство народного просвещения, в виду происходящих чрезвычайных событий, признало соответственным в день годовщины со дня рождения М.Ю. Лермонтова (2-го октября 1914 года) ограничиться совершением в учебных заведениях панихиды по поэту, отложив празднование юбилея до более благоприятного времени. «Русский инвалид», 16 сентября 1914 года."
2,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в районе Желтиева, в Галиции, летящий над нашим расположением австрийский аэроплан, собиравшийся бросить бомбы, взлетел на воздух, атаковал неприятеля и протаранил неприятельский аппарат, предотвратив жертвы в наших войсках. Сам Нестеров при этом погиб смертью героя. По словам доставленных в Киев пленных австрийских офицеров, всей неприятельской армии хорошо известно имя Нестерова. Во время воздушных разведок русских авиаторов австрийцы всегда безошибочно определяли, каким аппаратом управлял Нестеров. Когда показывался аэроплан-птица, красиво и вольно паривший в воздухе, австрийцы указывали — Das ist Nesteroff! Австрийцы боялись покойного, и все их усилия были направлены к прекращению его деятельности. За задержание отважного летчика была объявлена большая премия. Нестеров погиб в 27 лет. После Нестерова остались жена и двое детей — девочка, 5-ти лет, и мальчик, 3-х лет. Иллюстрированный журнал «Искры» № 35, сентябрь 1914 года Песнь о Нестерове В безбрежности неба, в бескрайности ясной пустыниСражались лишь птицы и гибли лишь птицы доныне.Так птице подобный, полет свой направив могучий,Унесся пилот легкокрылый за темные тучи.А там, на земле, колыхаясь, знамена шумели, И лязгали сабли, и пушки зловеще гремели.И пламенно богу войны посылал он молитвы,И очи горели и звали и жаждали битвы…Величие духа в равнине обманчиво-зыбкой.Презрение смерти под светлою солнца улыбкой…Мольбы не напрасны. Велению рока послушный,Противник уж реет кругами в стихии воздушнойИ дрогнуло сердце. Рука, как пружина стальная,На руль налегла. Встрепенулась машина живая…Чти, родина, память героя! Решенье созрелоМгновенье – и врезался дерзко он в хрупкое тело.Пощады не знает стихия. Победа наградой…И пали противники грузной и страшной громадой…В безбрежности неба, в бескрайности ясной пустыниСражались лишь птицы и гибли лишь птицы доныне. Журнал «Нива» №37, сентябрь 1914 года"
3,1914. Бульдог-гонец под Льежем,"Фотограф-корреспондент Daily Mirror рассказывает случай, который порадует всех друзей животных. Лейтенант бельгийской артиллерии, руководивший обороной одного из фортов Льежа, ни за что не хотел расстаться с своей собакой — бульдогом. Когда пруссаки пробрались между фортов в самый город, офицеру пришло в голову доверить бульдогу письмо, в котором он посылал успокоительную весть своим родителям. Благородный пес честно исполнил свою миссию. Десять часов спустя бульдог проник обратно в форт и принес ответ. С этого момента бульдог стал настоящим гонцом. Много раз пробирался он через линии германских войск, неся на себе спрятанными в ошейнике ш

In [20]:
df_news.shape

(800974, 2)

In [ ]:
df_train, df_test = train_test_split(df_news[:2000], # уменьшаем датасет
                                     test_size=0.2)

df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

dataset_train = Dataset.from_pandas(df_train)
dataset_test = Dataset.from_pandas(df_test)
dataset_train, dataset_test

In [24]:
model_name = "IlyaGusev/rut5_base_sum_gazeta"
tokenizer = AutoTokenizer.from_pretrained(model_name)

max_len_txt = 400
max_len_tlt = 50

def tokenize(batch):
    tokenized_input = tokenizer(batch['text'], padding='max_length', truncation=True, max_length=max_len_txt)
    tokenized_label = tokenizer(batch['title'], padding='max_length', truncation=True, max_length=max_len_tlt)
    tokenized_input['labels'] = tokenized_label['input_ids']

    return tokenized_input

dataset_train = dataset_train.map(tokenize, batched=True, batch_size=8)
dataset_test = dataset_test.map(tokenize, batched=True, batch_size=8)

dataset_train.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])
dataset_test.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [25]:
dataset_train.save_to_disk('lenta2/train')
dataset_test.save_to_disk('lenta2/test')

Saving the dataset (0/1 shards):   0%|          | 0/1600 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/400 [00:00<?, ? examples/s]

In [26]:
model_name = "IlyaGusev/rut5_base_sum_gazeta"
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [27]:
output_dir = 'lenta2/output'

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    save_steps=1000, 
    remove_unused_columns=True, 
    eval_steps=500, 
)

In [31]:
dataset_train = load_from_disk("lenta2/train")
dataset_test = load_from_disk("lenta2/test")
dataset_train, dataset_test

(Dataset({
     features: ['title', 'text', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 1600
 }),
 Dataset({
     features: ['title', 'text', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 400
 }))

In [32]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test
)

trainer.train()

Step,Training Loss
500,2.481300
1000,0.556600
1500,0.505300
2000,0.436000
2500,0.417500
3000,0.373200
3500,0.375600
4000,0.358400


TrainOutput(global_step=4000, training_loss=0.6879785842895508, metrics={'train_runtime': 855.9336, 'train_samples_per_second': 9.347, 'train_steps_per_second': 4.673, 'total_flos': 4248354816000000.0, 'train_loss': 0.6879785842895508, 'epoch': 5.0})

In [33]:
trainer.save_model(output_dir + '/model')

In [41]:
def pred(idx, model, dataset_test):
  input_text = dataset_test['text'][idx]
  input_title = dataset_test['title'][idx]

  device = torch.device("cuda:0") # ("cuda:0")

  with torch.no_grad():
    tokenized_text = tokenizer(input_text, 
                               truncation=True, 
                               padding=True, 
                               return_tensors='pt').to(device)

    source_ids = tokenized_text['input_ids'].to(dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(dtype = torch.long)

    generated_ids = model.generate(
        input_ids = source_ids,
        attention_mask = source_mask, 
        max_length=512,
        num_beams=10,
        temperature = 1.3,
        repetition_penalty=1, 
        length_penalty=1, 
        early_stopping=True,
        no_repeat_ngram_size=2
    ).to(device)

    pred = tokenizer.decode(generated_ids[0], 
                            skip_special_tokens=True, 
                            clean_up_tokenization_spaces=True)

  print("Настоящий заголовок: " + input_title)
  print("Сгенерированный заголовок: " + pred)
  print("Текст новости: " + input_text)

In [43]:
pred(0, model=model, dataset_test=dataset_test)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Настоящий заголовок: Рушайло: муфтий Кадыров — альтернативная фигура
Сгенерированный заголовок: Министр внутренних дел России считает, что Кадыров может стать "альтернативной фигурой" при переговорах между Путином и Масхадовым
Текст новости: Министр внутренних дел России Владимир Рушайло считает, что чеченский муфтий Ахмед Кадыров может стать "альтернативной фигурой" при переговорах между федеральным центром и Чечней по урегулированию ситуации в республике. Об этом он сообщил в четверг в беседе с журналистами, передает ИТАР-ТАСС. Глава МВД напомнил, что председатель правительства Владимир Путин и муфтий Кадыров встречались накануне и провели успешный диалог. Рушайло с удовлетворением отметил, что "политический диалог уже начался", однако он подчеркнул, что "мы пока не делаем далеко идущих выводов". Рушайло в очередной раз подчеркнул, что Аслан Масхадов не может представлять чеченскую сторону на переговорах. Напомним, что 11 октября Масхадов заявил, что он освобождает от должности муфти

In [44]:
pred(1, model=model, dataset_test=dataset_test)

Настоящий заголовок: Юрий Лужков: отставка Ельцина "достаточно реальна"
Сгенерированный заголовок: Лужков считает, что вариант проведения выборов в России неприемлем
Текст новости: Мэру Москвы Юрию Лужкову вероятность отставки президента и проведения в России досрочных президентских выборов кажется достаточно реальной. Как сообщает ИТАР-ТАСС, мэр заявил, что глава российского государства не может "по состоянию здоровья эффективно выполнять свои обязанности". В принципе, считает Лужков, вариант проведения досрочных президентских выборов вполне приемлем. Однако их совмещение по срокам с выборами в Государственную Думу недопустимо, поскольку избирателей "нельзя лишать права сделать выбор из более широкого круга претендентов". Вывод из игры "неугодных кандидатов" невозможен, отметил Юрий Лужков. Наоборот, в выборах главы государства необходимо участие всех претендентов. "В противном случае народ не будет иметь возможности сделать полноразмерный выбор", - добавил он. Мэр Москвы уверен: наиб

In [45]:
pred(2, model=model, dataset_test=dataset_test)

Настоящий заголовок: Венгерский суд постановил: Анатолий Быков будет содержаться под стражей в Будапеште
Сгенерированный заголовок: В Венгрии арестован Анатолий Быков
Текст новости: По вполне достоверным данным, поступившим к нам из редакции ежедневной газеты "Время МН", в пятницу 29 октября в Будапеште арестован красноярский предприниматель  Анатолий Быков, председатель совета директоров Красноярского алюминиевого завода (КрАЗа) и непримиримый противник губернатора Александра Лебедя. По сведениям корреспондента газеты "Время МН", в понедельник на судебном заседании в Будапеште (завершившемся около 20.00 по Москве) принято решение арестовать Анатолия Быкова и содержать его под стражей до предоставления российской стороной документов, подтверждающих обвинение г-на Быкова в организации заказных убийств на территории России. Подробности об истории задержания Быкова в Венгрии читайте в статье Владимира Шпака "Добрались" на страницах Интернет-газеты Вести.Ru и в его же специальном репортаже

In [46]:
pred(3, model=model, dataset_test=dataset_test)

Настоящий заголовок: Коммунистам разрешили участвовать в выборах
Сгенерированный заголовок: Центризбирком зарегистрировал список кандидатов в Госдуму
Текст новости: Центризбирком РФ зарегистрировал федеральный список кандидатов в депутаты Госдумы от Компартии РФ в количестве 255 человек. За недостоверные сведения, которые были признаны существенными, из него было исключено девять человек. Среди них только актриса Елена Дропека, которая стояла под 11-м номером,представляетобщефедеральную часть списка. Претензии к первой тройке Центризбирком и другие органы неимели. Еще порядка 20 кандидатов будут подвергнутыдополнительной проверке. Среди них Валентин Чикин, ПетрРоманов, Виталий Севастьянов и другие. У 17 кандидатовнедостоверные сведения признаны носящими несущественный характер. Первая же тройка списка коммунистов выглядит следующим образом: лидер КПРФ Геннадий Зюганов, председатель Государственной Думы Геннадий Селезнев и председатель Аграрного союза Российской Федерации  Василий Старо